In [ ]:
#pip install transformers trl accelerate torch bitsandbytes peft datasets -qU
#pip install scipy

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16,
)

In [2]:
MODEL_PATH = f"../models/MXLewd-L2-20B"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=nf4_config,
    device_map='auto',
    local_files_only=True,
    #trust_remote_code=False,
    use_cache=True
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "right"

In [7]:
def ask():
   global model
   messages = [
      {"role": "user", "content": "What is your favourite condiment?"},
      {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
      {"role": "user", "content": "Do you have mayonnaise recipes?"}
   ]
   messages = [
      {"role": "user", "content": "What are my value if I have 2 cards of 2, 3 in baccarat?"},
   ]
   encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
   model_inputs = encodeds.to(model.device)
   generated_ids = model.generate(model_inputs, 
                                  max_new_tokens=1000,
                                  pad_token_id=tokenizer.eos_token_id, 
                                  do_sample=True)
   decoded_output = tokenizer.batch_decode(generated_ids)
   #print(decoded_output[0])
   return decoded_output[0]

ask()
   

"<s> [INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\nWhat are my value if I have 2 cards of 2, 3 in baccarat? [/INST]\n\nThe value of your hand in Baccarat is determined by adding the face value of all the cards in your hand, except for Aces, which are worth 1, and 10s and face cards, which are worth 0.\n\nIn your hand, you have two cards of value 2 each, which gives you a total of 4. Also, you have a card of value 3, which makes your total score 7.\n\nYour hand value is 7.</s>"